### Q3. Import libraries and customer df

In [1]:
# import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# create path 

path = r'/Users/jonathanfava/07-2022 Instacart Basket Analysis'

In [4]:
# import new customers.csv

cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [11]:
# check df

cust.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [13]:
cust.shape

(206209, 10)

### Q4. Renamed columns. No drops

In [14]:
# rename columns

cust_2 = cust.rename(columns={'First Name': 'first_name', 'Surnam': 'last_name', 'Gender': 'gender', 'STATE': 'state', 'Age': 'age', 'n_dependants': '#_dependants', 'fam_status': 'marital_status'})

In [15]:
# check output

cust_2.head()

user_id first_name last_name  gender       state  age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   #_dependants marital_status  income  
0             3        married  165665  
1             0         single   59285  
2             2        married   99568  
3             0         single   42049  
4             1        married   40374

### Q5. Quality and consistency checks

In [10]:
# check descriptive statistics

cust_2.describe()

user_id            age   n_dependants         income
count  206209.000000  206209.000000  206209.000000  206209.000000
mean   103105.000000      49.501646       1.499823   94632.852548
std     59527.555167      18.480962       1.118433   42473.786988
min         1.000000      18.000000       0.000000   25903.000000
25%     51553.000000      33.000000       0.000000   59874.000000
50%    103105.000000      49.000000       1.000000   93547.000000
75%    154657.000000      66.000000       3.000000  124244.000000
max    206209.000000      81.000000       3.000000  593901.000000

In [17]:
# check for duplicates

cust_2[cust_2.duplicated()]

Empty DataFrame
Columns: [user_id, first_name, last_name, gender, state, age, date_joined, #_dependants, marital_status, income]
Index: []

In [20]:
# check for missing values

cust_2.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
#_dependants          0
marital_status        0
income                0
dtype: int64

In [21]:
# leaving customers with missing first name values because there are no duplicate last names.
# due to this each customer can still be differentiated. 

In [26]:
# check for mixed data types

for col in cust_2.columns.tolist():
   weird = (cust_2[[col]].applymap(type) != cust_2[[col]].iloc[0].apply(type)).any(axis = 1)
if len (cust_2[weird]) > 0:
        print(col)

In [27]:
# check data types

cust_2.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
#_dependants       int64
marital_status    object
income             int64
dtype: object

In [28]:
# double check df

cust_2.head()

user_id first_name last_name  gender       state  age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   #_dependants marital_status  income  
0             3        married  165665  
1             0         single   59285  
2             2        married   99568  
3             0         single   42049  
4             1        married   40374

### Q6. Combine customer df with ords_prods df

In [37]:
# import ords_prods df

ords_prods_1 = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_produts_4.9t.pkl'))

In [38]:
# find key column for merge

ords_prods_1.head()

product_id                product_name  aisle_id  department_id  prices  \
0           1  Chocolate Sandwich Cookies        61             19     5.8   
1           1  Chocolate Sandwich Cookies        61             19     5.8   
2           1  Chocolate Sandwich Cookies        61             19     5.8   
3           1  Chocolate Sandwich Cookies        61             19     5.8   
4           1  Chocolate Sandwich Cookies        61             19     5.8   

   order_id  user_id  order_number  order_day_of_week  time_of_order  ...  \
0   3139998      138            28                  6             11  ...   
1   1977647      138            30                  6             17  ...   
2    389851      709             2                  0             21  ...   
3    652770      764             1                  3             13  ...   
4   1813452      764             3                  4             17  ...   

     price_range_loc     busiest day  busiest_days  busiest_period_of_day  \
0  Mid range product  Regularly Busy  Slowest days            Most Orders   
1  Mid range product  Regularly Busy  Slowest days         Average Orders   
2  Mid range product     Busiest day  Busiest days         Average Orders   
3  Mid range product  Regularly Busy  Slowest days            Most Orders   
4  Mid range product      Least busy  Slowest days         Average Orders   

  max_order      loyalty_flag avg_spent spending_flag med_ord_freq  \
0        32  Regular customer  6.935811   Low spender          8.0   
1        32  Regular customer  6.935811   Low spender          8.0   
2         5      New customer  7.930208   Low spender          8.0   
3         3      New customer  4.972414   Low spender          9.0   
4         3      New customer  4.972414   Low spender          9.0   

      frequency_flag  
0  Frequent customer  
1  Frequent customer  
2  Frequent customer  
3  Frequent customer  
4  Frequent customer  

[5 rows x 26 columns]

In [39]:
# check user id data type in ords_prods df

ords_prods_1['user_id'].dtypes

dtype('int64')

In [33]:
# check user_id data type in cust_2 df

cust_2['user_id'].dtypes

dtype('int64')

In [40]:
# merge dfs on user_id column 

opc_merged = ords_prods_1.merge(cust_2, on = 'user_id')

In [41]:
# check output 

opc_merged.head()

product_id                product_name  aisle_id  department_id  prices  \
0           1  Chocolate Sandwich Cookies        61             19     5.8   
1           1  Chocolate Sandwich Cookies        61             19     5.8   
2         907        Premium Sliced Bacon       106             12    20.0   
3         907        Premium Sliced Bacon       106             12    20.0   
4        1000                    Apricots        18             10    12.9   

   order_id  user_id  order_number  order_day_of_week  time_of_order  ...  \
0   3139998      138            28                  6             11  ...   
1   1977647      138            30                  6             17  ...   
2   3160996      138             1                  5             13  ...   
3   2254091      138            10                  5             14  ...   
4    505689      138             9                  6             12  ...   

      frequency_flag  first_name  last_name  gender      state age  \
0  Frequent customer     Charles        Cox    Male  Minnesota  81   
1  Frequent customer     Charles        Cox    Male  Minnesota  81   
2  Frequent customer     Charles        Cox    Male  Minnesota  81   
3  Frequent customer     Charles        Cox    Male  Minnesota  81   
4  Frequent customer     Charles        Cox    Male  Minnesota  81   

  date_joined #_dependants marital_status income  
0    8/1/2019            1        married  49620  
1    8/1/2019            1        married  49620  
2    8/1/2019            1        married  49620  
3    8/1/2019            1        married  49620  
4    8/1/2019            1        married  49620  

[5 rows x 35 columns]

In [43]:
# check output

opc_merged.shape

(32404859, 35)

### Q8. Export merged df

In [44]:
# export as pickle

opc_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust_merged.pkl'))